# Numba Pitfalls

This is a collection of surprising behaviours of numba (at least as of version 0.61.2).

## Difference of Spec and Class Field Style: Holding other jitclass objects

In [5]:
from numba import int64
from numba.experimental import jitclass

inner_spec = [('v', int64)]
@jitclass(inner_spec)
class Inner:
    def __init__(self, v):
        self.v = v

@jitclass
class Outer:

    inner: Inner

    def __init__(self, inner):
        self.inner = inner


inner_obj = Inner(5)
outer_obj = Outer(inner_obj)
outer_obj.inner.v

5

In [6]:
from numba import int64
from numba.experimental import jitclass

inner_spec = [('v', int64)]
@jitclass(inner_spec)
class Inner:
    def __init__(self, v):
        self.v = v

outer_spec = [('inner', Inner.class_type)]
@jitclass(outer_spec)
class Outer:

    def __init__(self, inner):
        self.inner = inner

inner_obj = Inner(5)

# This will raise TypingError
outer_obj = Outer(inner_obj)
outer_obj.inner.v

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Failed in nopython mode pipeline (step: native lowering)
Cannot cast instance.jitclass.Inner#127b9fc50<v:int64> to jitclass.Inner#127b9fc50<v:int64>: %"inserted.data.1" = insertvalue {i8*, {i64}*} %"inserted.meminfo.1", {i64}* %"arg.inner.1", 1
During: lowering "(self).inner = inner" at /var/folders/zw/qxvhv2ms1rx684818_y1cvl40000gn/T/ipykernel_58903/3045205564.py (15)
During: Pass native_lowering
During: resolving callee type: jitclass.Outer#127d5fb60<inner:jitclass.Inner#127b9fc50<v:int64>>
During: typing of call at <string> (3)

During: resolving callee type: jitclass.Outer#127d5fb60<inner:jitclass.Inner#127b9fc50<v:int64>>
During: typing of call at <string> (3)


File "<string>", line 3:
<source missing, REPL/exec in use?>

During: Pass nopython_type_inference

## No functions as fields in jitclasses

In [10]:
from numba import types
from numba import njit

@njit
def some_njit_function(x):
    return 2*x

print(some_njit_function(10))

@jitclass
class Foo:

    f: types.FunctionType

    def __init__(self, f):
        self.f = f

20


TypingError: Cannot infer numba type of python type <class 'numba.core.types.function_type.FunctionType'>